# FEM para la ecuación de advección–difusión–reacción

En este notebook desarrollaremos la teoría de la formulación variacional y la implementación
del método de Elementos Finitos (FEM) con esquema θ para resolver la misma EDP en $\Omega=[0,1]^{2}$.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

# En notebook __file__ no existe, usamos cwd → notebooks/
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)

# Ajustamos path para importar nuestro src
import sys
root = BASE_DIR
if root not in sys.path:
    sys.path.insert(0, root)
